In [20]:
import sys
import os
import argparse
from multiprocessing import Process, Queue
from typing import List, Dict
import subprocess
import mmcv
import time
import csv
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time

In [12]:
parser = argparse.ArgumentParser()
parser.add_argument("--gpus", "-g", type=str, default="0",
                            help="space delimited GPU id list (global id in nvidia-smi, "
                                 "not considering CUDA_VISIBLE_DEVICES)")
parser.add_argument('--eval', action='store_true', default=False,
                   help='evaluation mode (run the render_imgs script)')
parser.add_argument('--configname',  default='syn_4096code',
                   help='a.k.a config  in subfloer ./configs/batchtest/<configname>.txt ')

parser.add_argument("--importance_include",  type=float,  default=0.6,
        help='fully vector quantize the full model')

parser.add_argument("--importance_prune",  type=float,  default=0.999,
        help='fully vector quantize the full model')

parser.add_argument("--codebook_size",  type=int,  default=4096,
        help='fully vector quantize the full model')

parser.add_argument("--dump_images",  action='store_true', default=False,
        help='fully vector quantize the full model')

parser.add_argument('--dataset', type=str, default='syn', choices=['syn', 'llff', 'tnt', 'nsvf'])
parser.add_argument('-f', type=str)
args = parser.parse_args()

PSNR_FILE_NAME = 'test_psnr.txt'

DatasetSetting={
    "syn": {
        "data": "./data/nerf_synthetic",
        "cfg": f"./configs/batch_test/{args.configname}.py",
        "basedir":f"./logs/{args.configname}",
        # "scene_list": ['chair'] #['chair', 'drums', 'ficus', 'hotdog', 'lego', 'materials', 'mic', 'ship']
        "scene_list": ['drums', 'ficus', 'hotdog', 'lego', 'materials', 'mic', 'ship']
    },
    "tnt":{
        "data": "./data/TanksAndTemple",
        "cfg": f"./configs/batch_test/{args.configname}.py",
        "basedir":f"./logs/{args.configname}",
        "scene_list":['Barn', 'Caterpillar', 'Family', 'Ignatius', 'Truck']
    },
    "nsvf":{
        "data": "./data/Synthetic_NSVF",
        "cfg": f"./configs/batch_test/{args.configname}.py",
        "basedir":f"./logs/{args.configname}",
        "scene_list":['Bike', 'Lifestyle', 'Palace', 'Robot', 'Spaceship', 'Steamtrain', 'Toad', 'Wineholder',]
    }
    
}

datasetting = DatasetSetting[args.dataset]

class AverageMeter(object):
    def __init__(self, name=''):
        self.name=name
        self.reset()
    def reset(self):
        self.val=0
        self.sum=0
        self.avg=0
        self.count=0
    def update(self,val,n=1):
        self.val=val
        self.sum += val*n
        self.count += n
        self.avg=self.sum/self.count
    def __repr__(self) -> str:
        return f'{self.name}: average {self.count}: {self.avg}\n'

from prettytable import PrettyTable




In [15]:

columns = ["exp_name", "Scene", 'netdepth', 'netwidth', "PSNR", "SSIM", "LPIPS_ALEX","LPIPS_VGG", "SIZE", "eps_coarse","eps_fine","eps_vq","eps_time"]
df = pd.DataFrame(columns=columns)
df = df.set_index('exp_name')

for depth in range(3,9):
    width = 128
    
    table = PrettyTable(["Scene", "PSNR", "SSIM", "LPIPS_ALEX", "LPIPS_VGG", "SIZE", "TIME"])
    table.float_format = '.3'
    
    PSNR=AverageMeter('PSNR')
    SSIM=AverageMeter('SSIM')
    LPIPS_A=AverageMeter('LPIPS_A')
    LPIPS_V=AverageMeter('LPIPS_V')
    SIZE=AverageMeter('SIZE')

    for scene in datasetting["scene_list"]: #[ 'chair', 'drums', 'ficus', 'hotdog', 'lego', 'mic', 'materials', 'ship'   ]:
        
        path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/time.csv'
        with open(path, 'r') as f:
            times = [r for r in csv.DictReader(f)][0]
            timestrs = 
            x = time.strptime('00:01:00','%H:%M:%S')
datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()

        path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/render_test_vq_last/mean.txt'
        with open(path, 'r') as f:
            lines = f.readlines()
            psnr = float(lines[0].strip())
            ssim = float(lines[1].strip())
            lpips_a = float(lines[2].strip())
            lpips_v = float(lines[3].strip())
            PSNR.update(psnr)
            SSIM.update(ssim)
            LPIPS_A.update(lpips_a)
            LPIPS_V.update(lpips_v)
            compressed_file = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/extreme_saving.zip'
            if os.path.exists(compressed_file):
                size = os.path.getsize(compressed_file)/(1024*1024)
            else:
                size = 0
            table.add_row([scene, psnr, ssim, lpips_a, lpips_v, size, times['eps_time']])
            SIZE.update(size)
            exp_name = f'{scene}_d{depth}_w{width}'
            data = {
                'exp_name': exp_name,
                'Scene': scene,
                'netdepth': depth, 'netwidth': width, "PSNR": psnr, "SSIM": ssim, "LPIPS_ALEX": lpips_a,
                "LPIPS_VGG": lpips_v, "SIZE": size, **times,
            }
            
            df.loc[exp_name] = data
    table.add_row(['Mean', PSNR.avg, SSIM.avg, LPIPS_A.avg,LPIPS_V.avg, SIZE.avg, "--"])

    txt_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_{args.importance_include}_{args.importance_prune}_{args.codebook_size}.txt')
    with open(txt_file, 'w') as f:
        f.writelines(table.get_string())

    csv_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_{args.importance_include}_{args.importance_prune}_{args.codebook_size}.csv')
    with open(csv_file, 'w', newline='') as f:
        f.writelines(table.get_csv_string())
    # print('VQ-DVGO:')
    # print(table)


#     table = PrettyTable(["Scene", "PSNR", "SSIM", "LPIPS_ALEX","LPIPS_VGG", "SIZE", "TIME"])
#     table.float_format = '.3'

#     PSNR=AverageMeter('PSNR')
#     SSIM=AverageMeter('SSIM')
#     LPIPS_A=AverageMeter('LPIPS_A')
#     LPIPS_V=AverageMeter('LPIPS_V')
#     SIZE=AverageMeter('SIZE')


#     for scene in datasetting["scene_list"]: #[ 'chair', 'drums', 'ficus', 'hotdog', 'lego', 'mic', 'materials', 'ship'   ]:
#         path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/time.csv'
#         with open(path, 'r') as f:
#             times = [r for r in csv.DictReader(f)][0]
            
#         path = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/render_test_fine_last/mean.txt'

#         with open(path, 'r') as f:
#             lines = f.readlines()
#             psnr = float(lines[0].strip())
#             ssim = float(lines[1].strip())
#             lpips_a = float(lines[2].strip())
#             lpips_v = float(lines[3].strip())
#             PSNR.update(psnr)
#             SSIM.update(ssim)
#             LPIPS_A.update(lpips_a)
#             LPIPS_V.update(lpips_v)
#             compressed_file = f'./logs/{args.configname}_d{depth}_w{width}/{args.configname}_{scene}/fine_model.pth.zip'
#             if os.path.exists(compressed_file):
#                 size = os.path.getsize(compressed_file)/(1024*1024)
#             else:
#                 size = 0
#             table.add_row([scene, psnr, ssim, lpips_a, lpips_v, size, times['eps_fine']])
#             SIZE.update(size)
#     table.add_row(['Mean', PSNR.avg, SSIM.avg, LPIPS_A.avg,LPIPS_V.avg, SIZE.avg, 0])

#     txt_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_fine.txt')
#     with open(txt_file, 'w') as f:
#         f.writelines(table.get_string())

#     csv_file = os.path.join(datasetting["basedir"] + f"_d{depth}_w{width}", f'merge_fine.csv')
#     with open(csv_file, 'w', newline='') as f:
#         f.writelines(table.get_csv_string())
#     print('orginal DVGO:')
#     print(table)

df

,Scene,netdepth,netwidth,PSNR,SSIM,LPIPS_ALEX,LPIPS_VGG,SIZE,eps_coarse,eps_fine,eps_vq,eps_time
exp_name,,,,,,,,,,,,
drums_d3_w128,drums,3,128,25.380116,0.927488,0.082850,0.061382,0.895280,00:00:46,00:02:27,00:03:11,00:06:24
ficus_d3_w128,ficus,3,128,32.707341,0.976990,0.027883,0.017132,1.061307,00:00:39,00:02:43,00:03:10,00:06:31
hotdog_d3_w128,hotdog,3,128,36.494715,0.978110,0.038983,0.018611,1.457404,00:00:43,00:03:24,00:02:59,00:07:05
lego_d3_w128,lego,3,128,34.281551,0.973460,0.029738,0.013213,1.483098,00:00:44,00:02:50,00:02:50,00:06:23
materials_d3_w128,materials,3,128,29.265209,0.945007,0.066244,0.032863,3.099672,00:00:40,00:03:32,00:03:18,00:07:30
mic_d3_w128,mic,3,128,33.124954,0.982265,0.020384,0.013882,0.410399,00:00:46,00:01:42,00:03:04,00:05:32
ship_d3_w128,ship,3,128,29.152933,0.876825,0.160234,0.112917,2.059130,00:00:42,00:04:40,00:04:17,00:09:40
drums_d4_w128,drums,4,128,25.410795,0.929342,0.080410,0.059036,0.916064,00:00:46,00:02:54,00:03:23,00:07:02
ficus_d4_w128,ficus,4,128,33.048894,0.978821,0.026013,0.015717,1.082466,00:00:39,00:03:18,00:03:45,00:07:42


In [ ]:

fig, axes = plt.subplots(2, 4, figsize=(20,10))

60.0